<table style="width:100%; border-collapse: collapse;">
  <tr>
    <td style="width:20%; vertical-align:middle;">
      <img src="LogoUVG.png" width="400"/>
    </td>
    <td style="text-align:left; vertical-align:middle;">
      <h2 style="margin-bottom: 0;">Universidad del Valle de Guatemala - UVG</h2>
      <h3 style="margin-top: 0;">Facultad de Ingeniería - Computación</h3>
      <p style="font-size: 16px; margin-bottom: 0; margin-top: -20px">
        <strong>Curso:</strong> Visión por Computadora 
        <strong>Sección:</strong> 10
      </p>
      <p style="font-size: 16px; margin: 0;"><strong>Proyecto 2:</strong> Detección en video e imágenes</p>
      <br>
      <p style="font-size: 15px; margin: 0;"><strong>Autores:</strong></p>
      <ul style="margin-top: 5px; padding-left: 20px; font-size: 15px;">
        <li>Diego Alexander Hernández Silvestre - <strong>21270</strong></li>
        <li>Linda Inés Jiménez Vides - <strong>21169</strong></li>
        <li>José Andrés Auyón Cobar - <strong>201579</strong></li>
      </ul>
    </td>
  </tr>
</table>


#### Librerias 📚

In [3]:
import cv2
import mediapipe as mp
import time

In [4]:
mpHands = mp.solutions.hands
mpDrawing = mp.solutions.drawing_utils
mpStyles = mp.solutions.drawing_styles
prevTime = 0
hands = mpHands.Hands(
    static_image_mode=False,
    max_num_hands=8,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgbImage = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgbImage.flags.writeable = False
    results = hands.process(rgbImage)
    rgbImage.flags.writeable = True
    bgrImage = cv2.cvtColor(rgbImage, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for handLandmarks in results.multi_hand_landmarks:
            mpDrawing.draw_landmarks(
                image=bgrImage,
                landmark_list=handLandmarks,
                connections=mpHands.HAND_CONNECTIONS,
                landmark_drawing_spec=mpStyles.get_default_hand_landmarks_style(),
                connection_drawing_spec=mpStyles.get_default_hand_connections_style()
            )
            h, w, _ = bgrImage.shape
            xMin, yMin = w, h
            xMax, yMax = 0, 0

            for lm in handLandmarks.landmark:
                x, y = int(lm.x * w), int(lm.y * h)
                xMin = min(xMin, x)
                yMin = min(yMin, y)
                xMax = max(xMax, x)
                yMax = max(yMax, y)

            cv2.rectangle(bgrImage, (xMin - 20, yMin - 20), (xMax + 20, yMax + 20), (0, 255, 100), 2, lineType=cv2.LINE_AA)

    currTime = time.time()
    fps = 1 / (currTime - prevTime) if (currTime - prevTime) > 0 else 0
    prevTime = currTime
    windowTitle = f'I. Deteccion de manos y keypoints - FPS: {int(fps)}'
    cv2.setWindowTitle('main', windowTitle)
    bgrImage = cv2.resize(bgrImage, (1280, 960))
    cv2.imshow('main', bgrImage)
    cv2.resizeWindow('main', 1280, 960)
    if cv2.waitKey(5) & 0xFF == ord('e'):
        break

cap.release()
cv2.destroyAllWindows()